In [4]:
#Customer Lifetime Value (CLV) Analysis
import pandas as pd
df = pd.read_csv("cleaned_online_retail.csv", parse_dates=["InvoiceDate"])
print( df.columns.tolist())
df["TotalPrice"] = df["Quantity"] * df["Price"]

# Group by Customer ID and calculate basic CLV metrics
customer_metrics = df.groupby("Customer ID").agg({
    "Invoice": "nunique",       # Total number of orders
    "InvoiceDate": [min, max],  # First and last purchase
    "TotalPrice": "sum"         # Total money spent
})

# Rename columns
customer_metrics.columns = ["TotalOrders", "FirstPurchaseDate", "LastPurchaseDate", "TotalRevenue"]
customer_metrics = customer_metrics.reset_index()

# Calculate average order value
customer_metrics["AvgOrderValue"] = customer_metrics["TotalRevenue"] / customer_metrics["TotalOrders"]

# Calculate number of days as customer
customer_metrics["CustomerAgeDays"] = (customer_metrics["LastPurchaseDate"] - customer_metrics["FirstPurchaseDate"]).dt.days
customer_metrics["CustomerAgeDays"] = customer_metrics["CustomerAgeDays"].replace(0, 1)  # avoid division by 0

# Orders per day
customer_metrics["OrdersPerDay"] = customer_metrics["TotalOrders"] / customer_metrics["CustomerAgeDays"]

# Simple CLV = AOV * Orders Per Day * 30 (to get monthly value)
customer_metrics["CLV"] = customer_metrics["AvgOrderValue"] * customer_metrics["OrdersPerDay"] * 30

# top 10 high-value customers
top_customers = customer_metrics.sort_values(by="CLV", ascending=False).head(10)

print("Top 10 Customers by CLV:")
print(top_customers[["Customer ID", "CLV", "TotalRevenue", "AvgOrderValue"]])





Columns available: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'TotalPrice']


C:\Users\91994\AppData\Local\Temp\ipykernel_12180\2046496565.py:14: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  customer_metrics = df.groupby("Customer ID").agg({
C:\Users\91994\AppData\Local\Temp\ipykernel_12180\2046496565.py:14: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  customer_metrics = df.groupby("Customer ID").agg({


Top 10 Customers by CLV:
      Customer ID        CLV  TotalRevenue  AvgOrderValue
2444      15760.0  417490.20     13916.340    6958.170000
9         12357.0  362399.70     12079.990    6039.995000
933       13687.0  356425.20     11880.840   11880.840000
374       12918.0  328605.00     10953.500   10953.500000
4144      18052.0  326315.40     10877.180   10877.180000
1179      14028.0  311895.00     10396.500    3465.500000
2050      15202.0  180044.70      6001.490    2000.496667
2712      16118.0  131287.50      4376.250    4376.250000
242       12737.0  111315.00      3710.500    1855.250000
1491      14459.0  106134.63      3537.821    3537.821000
